# Polarization on Twitter
### By Jonathan Gustafsson Frennert

### I. Dependencies

In [1]:
import os

import matplotlib.pyplot as plt
import matplotlib as mpl
from mpl_toolkits import mplot3d
import seaborn as sns

import math
import numpy as np
np.random.seed(42)
from scipy.stats import mstats
import pandas as pd
import dask.dataframe as dd

import warnings
warnings.filterwarnings('ignore')

print("All packages imported!")

All packages imported!


### II. Matplotlib Parameters

In [2]:
mpl.rcParams['figure.dpi'] = 100
mpl.rcParams['font.size'] = 9

In [3]:
# Latex document Text width
latex_width = 390.0

def set_size(width=latex_width, height=latex_width, fraction=1, subplots=(1, 1)):
    """Set figure dimensions to avoid scaling in LaTeX.
    
    Credit to Jack Walton for the function.
    Source: https://jwalton.info/Embed-Publication-Matplotlib-Latex/
    """

    fig_width_pt = width * fraction
    fig_height_pt = height * fraction
    
    inches_per_pt = 1 / 72.27
    
    fig_width_in = fig_width_pt * inches_per_pt
    fig_height_in = fig_height_pt * inches_per_pt * (subplots[0] / subplots[1])

    return (fig_width_in, fig_height_in)

## III. Color Palette

The palette is from the [iWantHue](http://medialab.github.io/iwanthue/) website by Mathieu Jacomy at the Sciences-Po Medialab.

In [4]:
colors = [
    "#ba4c40",
    "#45c097",
    "#573485",
    "#a8ae3e",
    "#8874d9",
    "#69a050",
    "#be64b2",
    "#bc7d36",
    "#5d8ad4",
    "#b94973"
]

## IV. Twitter Dataset

**Provenance:** Ibrahim Sabuncu, "USA Nov.2020 Election 20 Mil. Tweets (with Sentiment and Party Name Labels) Dataset." *IEEE Dataport*, 14 Aug. 2020, doi: https://dx.doi.org/10.21227/25te-j338.

**License:** [Developer Agreement](https://developer.twitter.com/en/developer-terms/agreement)

**Usage Information:** 
- "you may only use the following information for non-commercial, internal purposes (e.g., to improve the functionality of the Services): (a) aggregate Twitter Applications user metrics, such as number of active users or accounts on Twitter Applications; (b) the responsiveness of Twitter Applications; and (c) results, usage statistics, data or other information (in the aggregate or otherwise) derived from analyzing, using, or regarding the performance of the Twitter API."

- "you may not use, or knowingly display, distribute, or otherwise make Twitter Content, or information derived from Twitter Content, available to any entity for the purpose of: (a) conducting or providing surveillance or gathering intelligence, including but not limited to investigating or tracking Twitter users or Twitter Content; (b) conducting or providing analysis or research for any unlawful or discriminatory purpose, or in a manner that would be inconsistent with Twitter users' reasonable expectations of privacy; (c) monitoring sensitive events (including but not limited to protests, rallies, or community organizing meetings); or (d) targeting, segmenting, or profiling individuals based on sensitive personal information, including their health (e.g., pregnancy), negative financial status or condition, political affiliation or beliefs, racial or ethnic origin, religious or philosophical affiliation or beliefs, sex life or sexual orientation, trade union membership, Twitter Content relating to any alleged or actual commission of a crime, or any other sensitive categories of personal information prohibited by law."

**Sentiment analysis tool:** Hutto, C.J. & Gilbert, E.E. (2014). "VADER: A Parsimonious Rule-based Model for Sentiment Analysis of Social Media Text." *Eighth International Conference on Weblogs and Social Media (ICWSM-14)*, June 2014. [GitHub](https://github.com/cjhutto/vaderSentiment)

<center> <h3>Dataset Contents*</h3> </center>

<center> <h4><code>uselection_tweets_1jul_11nov.csv</code></h4> </center>

| Variable | Format | Description | Example |
| :- | :- | :- | :- | 
| `Created-At`$\,$ | Timestamp$\,$ | Time at which tweet was created $\,$ | 7/1/20 7:44 PM |
| `From-User-Id`$\,$ | String$\,$ | Unique identifier of the user that sent the tweet $\,$ | 1223446325758394369 |
| `To-User-Id`$\,$ | String$\,$ | Unique identifier of the user that tweet sent to, -1 if nobody $\,$ | 387882597 |
| `Language`$\,$ | String$\,$ | ISO 639-1 language of the tweet $\,$ | en |
| `PartyName`$\,$ | String$\,$ | Which party is mentioned in the tweet $\,$ | BothParty |
| `Id`$\,$ | String$\,$ | Unique identifier of the tweet $\,$ | 1278368973948694528 |
| `Score`$\,$ | Float$\,$ | The sentiment score of the tweet $\,$ | 0.102564 |
| `Scoring String`$\,$ | String$\,$ | The sentiment score of the tweet $\,$ | 0.102564 |

\**only imported fields are shown.*

### Importing

In [5]:
twitter_cols = ['Created-At', 'From-User-Id', 'To-User-Id', 'Language', 'PartyName', 'Id', 'Score', 'Scoring String']
twitter_filepath = os.path.join(os.getcwd(), 'data', 'twitter', 'uselection_tweets_1jul_11nov.csv')

# twitter dask dataframe
tdd = dd.read_csv(twitter_filepath, sep=';', usecols=twitter_cols)

### Cleaning

- No duplicates found.

- Extreme values were found, but were kept.

#### Correcting Inferred Variable Types

In [6]:
tdd.dtypes

Created-At         object
From-User-Id        int64
To-User-Id          int64
Language           object
PartyName          object
Id                  int64
Score             float64
Scoring String     object
dtype: object

- `Created-At` should be a timestamp
- `From-User_Id` should be a string
- `To-User_Id` should be a string
- `Id` should be a string

In [7]:
tdd['Created-At'] = dd.to_datetime(tdd['Created-At'])
tdd['From-User-Id'] = tdd['From-User-Id'].astype('str')
tdd['To-User-Id'] = tdd['To-User-Id'].astype('str')
tdd['Id'] = tdd['Id'].astype('str')

#### Remove `NaN` Values

Some tweets were unscored and hence the `Scoring String` is `NaN`. These tweets were all removed.

In [8]:
tdd = tdd.dropna()

#### Initial Filters

- We filter for English to get more relevant tweets; the US speaks English. Further, the experiment done later involved English Speakers.

- We are primarily interested in tweets that can be fitted along the conservative-democrat political axis.

In [9]:
tdd = tdd[tdd['Language'] == 'en']
tdd = tdd[(tdd['PartyName'] == 'Republicans') | (tdd['PartyName'] == 'Democrats')]

#### Political Polarity

- We convert the categories into numerical values by sign.

In [10]:
polarity = {'Republicans': -1, 'Democrats' : 1}

tdd['PartyName'] = tdd['PartyName'].replace(polarity)

#### Drop Unused Columns

In [11]:
tdd = tdd.drop(columns=['Language', 'Scoring String'])

#### Rename and Reorder Columns

No particular reason, just prefer not using dictionary to access dataframe columns and a certain order of columns.

In [12]:
old_to_new = {
    'Created-At'   : 'time',
    'Id'           : 'id',
    'From-User-Id' : 'from',
    'To-User-Id'   : 'to',
    'PartyName'    : 'party',
    'Score'        : 'emotion'
}

order = ['time', 'id', 'from', 'to', 'party', 'emotion']

tdd = tdd.rename(columns=old_to_new)[order]

#### Compute Changes

Compute all cleaning at once and get a **Pandas** Dataframe.

In [13]:
tdf = tdd.compute()

## 1 Political Opinion of Tweets

### 1.1 How well does `party` and `emotion` capture Political Opinion? 

To test the accuracy of the metrics, we will take a random sample of 1000 tweets and have I and a non-author assign the tweets to the four quadrants made up of party and emotion axis. We will then apply hypothesis testing on the accuracy to confirm that the results are not up to chance (binomial distribution).

In [14]:
def quadrant(row):
    p = row.party
    e = row.emotion
    
    if (e > 0) and (p == 1):
        result = 1
    elif (e < 0) and (p == -1):
        result = 2
    elif (e < 0) and (p == 1):
        result = 3
    elif (e > 0) and (p == -1):
        result = 4
    else:
        result = 0
    
    return result

#### Creating sample

Retrieving 2000 ids as some tweets may be privated, deleted etc...

In [30]:
#file = 'sample.csv'
#tdf.id.sample(2000).to_csv(file, header = False, index = False)

#### Exporting Hydrated Sample

In [31]:
#sample_cols = ['id', 'text']
#sample_filepath = os.path.join(os.getcwd(), 'data', file)
#sdf = pd.read_csv(sample_filepath, usecols=sample_cols, dtype=str)

# prepare experiment
#sdf = sdf.dropna().head(1000)
#sdf = sdf.merge(right=tdf, how='left', on='id')
#sdf['label'] = sdf.apply(quadrant, axis=1)
#sdf = sdf[['id', 'text', 'label']]


#experiment_filepath = os.path.join(os.getcwd(), 'data', 'experiment.csv')
#sdf.to_csv(experiment_filepath, index=False)

#### Import Experiment Data

### 1.2 Political Opinion Metric

The political opinion of a tweet is the *point along the conservative-democrat political axis $[-1,1]$ that the content in the tweet expresses*. We want to convert tweets to these values as it lets us quantitively study political polarization over the time period the tweets were collected. 

#### `party` x Linearly Scaled `emotion`

We will be scaling the `emotion` to $[-1,1]$ using minimum and maximum `emotion` values, however we do not want the outliers to have a disproportionate effect. To fix this, we will winzorise the emotion scores. Intuitively, it is unlikely that outliers are expressing particularly more emotions than the 0.01 percentile or more than the 99.99 percentile (~2500 tweets). These tweets are largely result from people repeating a certain connotated word (e.g., love or loser) that is picked up by the sentiment analysis tool leading to extreme values.

In [ ]:
xs = mstats.winsorize(tdf.emotion, limits=[0.0001, 0.0001]) 

# maximum magnitude of emotion
scale = max(-xs.min(), xs.max())

# political opinion metric
tdf['opinion'] = (tdf.party * xs / scale).values

In [ ]:
sns.histplot(tdf.opinion.sample(5000))